In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# Build the dataset

N = 2000
norm00 = np.random.multivariate_normal([-3, -3], [[1, 0], [0, 1]], size=N // 2)
norm11 = np.random.multivariate_normal([3, 3], [[1, 0], [0, 1]], size=N // 2)
labels_np = np.int64(np.hstack((np.zeros(norm00.shape[0]),
                                np.ones(norm11.shape[0])
                                )))
features_np = np.float32(np.vstack((norm00, norm11)))
data = np.hstack([features_np, labels_np[:, None]])
np.random.shuffle(data)

labels_np, features_np = np.intp(data[:, -1]), np.float32(data[:, :2])

cut = N * 2 // 3
features_training_np, labels_training_np = features_np[:cut], labels_np[:cut]
features_testing_np, labels_testing_np = features_np[cut:], labels_np[cut:]

In [ ]:
# Generate the fingerprints
FP_NUM = 4
EPS = 0.06
alpha = 0.25
beta = 0.75
NUM_CLASS = 2
dx = (np.random.rand(FP_NUM, features_np.shape[1]) - .5) * 2 * EPS

dy_train = -np.ones((NUM_CLASS, NUM_CLASS)) * alpha  # 2 is number of classes
for i in range(NUM_CLASS):
    dy_train[i, i] = beta

In [ ]:
# Define the models
def nn(input_shape):
    model = tf.keras.Sequential((
        tf.keras.layers.Dense(200, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASS, activation=None)))  # not Softmax
    model.build()
    return model

In [ ]:
# Build the graph
features_tf = tf.placeholder(features_np.dtype, [None, 2])
labels = tf.placeholder(labels_np.dtype, [None])
dx_tf = tf.placeholder(features_np.dtype, [FP_NUM, None, 2])
dy_tf = tf.constant(dy_train, dtype=dx_tf.dtype)

labels_oh = tf.one_hot(labels, NUM_CLASS)


logits = nn(features_np.shape)(features_tf)

loss_vanilla = tf.losses.softmax_cross_entropy(labels_oh, logits)
probs = tf.nn.softmax(logits)
classification = tf.argmax(probs, axis=1)
correct_classification = tf.equal(classification, labels)
# TODO MAYBE USE tf.accuracy?
accuracy = tf.reduce_mean(tf.cast(correct_classification, tf.float32))

fp_logits = nn(features_np.shape)(features_tf + dx_tf)


def normalize(fp_logits):
    return fp_logits / tf.norm(fp_logits, axis=1)[:, None]


# TODO PIETRO -> orderd should be inverted..
FxDx = normalize(logits) - normalize(fp_logits)


Dy = tf.gather(dy_tf, labels)
Dy = tf.stack([Dy] * FP_NUM)

loss_fp = tf.losses.mean_squared_error(Dy, FxDx)
loss = loss_vanilla + loss_fp
train = tf.train.AdamOptimizer(0.001).minimize(loss)


sess = tf.Session()
sess.run(tf.global_variables_initializer())


training_dict = {features_tf: features_training_np,
                 labels: labels_training_np,
                 dx_tf: dx[:, np.newaxis, :]}

testing_dict = {features_tf: features_testing_np,
                labels: labels_testing_np,
                dx_tf: dx[:, np.newaxis, :]}


for epoch in range(500):
    sess.run([train], feed_dict=training_dict)
    test_data_loss, test_fp_loss, test_loss, test_acc = sess.run(
        [loss_vanilla, loss_fp, loss, accuracy],
        feed_dict=testing_dict
    )
    if epoch % 10 == 0:
        print('epoch', epoch, 'data loss %.6f fp loss %.6f total loss: %.6f accuracy: %.3f' % (
            test_data_loss, test_fp_loss, test_loss, test_acc))

In [ ]:
x, y = np.mgrid[-8:8:0.1, -8:8:0.1]
prob_c = sess.run(probs > 0.5, feed_dict={
                  features_tf: np.dstack((x, y)).reshape((-1, 2))})[:, 1]
plt.contourf(x, y, prob_c.reshape(x.shape), cmap='gray')
plt.colorbar()
plt.scatter(features_testing_np[:, 0],
            features_testing_np[:, 1], c=labels_testing_np)
for i in range(FP_NUM):
    plt.arrow(-3, -3, 100 * dx[i, 0], 100 * dx[i, 1], color='r')
plt.show()

In [ ]:
t1 = tf.expand_dims(FxDx, 2)
t2 = t1 - dy_tf
t3 = t2 ** 2
t4 = tf.reduce_sum(t3, axis=-1)
t5 = (1.0 / FP_NUM) * tf.sqrt(t4)
t6 = tf.transpose(t5, [1, 2, 0])
t7 = tf.reduce_sum(t6, axis=-1)
t8 = tf.reduce_min(t7, axis=-1)

x, y = np.mgrid[-8:8:0.05, -8:8:0.05]
dissimilarities = sess.run(t8, feed_dict={features_tf: np.dstack(
    (x, y)).reshape((-1, 2)), dx_tf: dx[:, None, :]})

In [ ]:
plt.contourf(x, y, dissimilarities.reshape(x.shape), cmap='gray')
plt.colorbar()
plt.scatter(features_testing_np[:, 0],
            features_testing_np[:, 1], c=labels_testing_np)
plt.show()

In [ ]:
# VERIFY FINGERPRINTS

mini_dict = {
    features_tf: features_testing_np[:10],
    labels: labels_testing_np[:10],
    dx_tf: dx[:, np.newaxis, :]
}
fxdx, dy = sess.run([FxDx, Dy], feed_dict=mini_dict)


i = 6
print(dy[:, i])
print(fxdx[:, i])
print(labels_testing_np[0])